In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import taichi as ti
from dataclasses import dataclass
from typing import List, Tuple

# Configuration matplotlib pour affichage en ligne
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 10

## Étape 1 : Jeux de Données (Configurations)



In [ ]:
@dataclass
class SimulationConfig:
    """Configuration paramétrique pour une simulation"""
    name: str
    N: int  # Nombre de boids
    WIDTH: float  # Largeur du domaine
    HEIGHT: float  # Hauteur du domaine
    dt: float  # Pas de temps
    NEIGHBOR_RADIUS: float  # Rayon de perception (cohésion)
    DESIRED_SEPARATION: float  # Distance minimale désirée (séparation)
    ALIGNMENT_RADIUS: float  # Rayon pour l'alignement
    MAX_SPEED: float  # Vitesse maximale
    MAX_FORCE: float  # Force maximale appliquée
    W_SEPARATION: float  # Poids de la séparation
    W_ALIGNMENT: float  # Poids de l'alignement
    W_COHESION: float  # Poids de la cohésion
    T: int  # Nombre d'itérations

# Configurations du fichier original - 3 presets pour différents comportements
CONFIGS = {
    "cohesive": SimulationConfig(
        name="Cohésive", N=200, WIDTH=35.0, HEIGHT=35.0, dt=0.1,
        NEIGHBOR_RADIUS=5.0, DESIRED_SEPARATION=1.5, ALIGNMENT_RADIUS=4.0,
        MAX_SPEED=1.5, MAX_FORCE=0.08,
        W_SEPARATION=1.8, W_ALIGNMENT=1.2, W_COHESION=1.0, T=1500
    ),
    "fragmented": SimulationConfig(
        name="Fragmentée", N=160, WIDTH=45.0, HEIGHT=45.0, dt=0.1,
        NEIGHBOR_RADIUS=3.5, DESIRED_SEPARATION=2.0, ALIGNMENT_RADIUS=2.8,
        MAX_SPEED=1.4, MAX_FORCE=0.08,
        W_SEPARATION=2.0, W_ALIGNMENT=0.8, W_COHESION=0.4, T=1500
    ),
    "chaotic": SimulationConfig(
        name="Chaotique", N=220, WIDTH=55.0, HEIGHT=55.0, dt=0.1,
        NEIGHBOR_RADIUS=2.5, DESIRED_SEPARATION=2.5, ALIGNMENT_RADIUS=2.0,
        MAX_SPEED=1.2, MAX_FORCE=0.06,
        W_SEPARATION=2.0, W_ALIGNMENT=0.4, W_COHESION=0.1, T=1500
    )
}

# Affichage des configurations disponibles avec leurs densités
print("Configurations disponibles:")
for key, cfg in CONFIGS.items():
    print(f"  {key}: N={cfg.N}, densité={cfg.N/(cfg.WIDTH*cfg.HEIGHT):.2f} boids/unité²")

## Étape 2 : Implémentation NumPy du Modèle de Reynolds

### Les 3 Principes :
1. **Séparation** : Éviter les collisions avec les voisins proches
2. **Alignement** : Aligner sa vélocité avec celle des voisins
3. **Cohésion** : Se diriger vers le centre de masse du groupe

In [ ]:
class BoidSimulationNumPy:
    """Simulation des boids en NumPy pur - Version de base sans obstacles"""
    
    def __init__(self, config: SimulationConfig, interaction_mode: str = "linear"):
        self.config = config
        self.interaction_mode = interaction_mode  # "linear" ou "quadratic"
        
        # Initialisation aléatoire uniforme sur un rectangle (topologie torique)
        self.pos = np.column_stack([
            np.random.uniform(0, config.WIDTH, config.N),
            np.random.uniform(0, config.HEIGHT, config.N)
        ])
        # Vitesses initiales aléatoires
        self.vel = np.random.uniform(-1, 1, (config.N, 2))
        # Accélérations (utilisées dans certains modèles)
        self.acc = np.zeros((config.N, 2))
        
    def _limit_vec(self, vec, max_val):
        """Limiter la norme des vecteurs à max_val
        """
 
        norms = np.linalg.norm(vec)
    
        return  vec / norms * max_val if norms>max_val else vec
    
    def _wrap_position(self, pos):
        """Périodique sur rectangle (topologie torique)."""
        pos_wrapped = pos.copy()
        pos_wrapped[:, 0] = np.mod(pos_wrapped[:, 0], self.config.WIDTH)
        pos_wrapped[:, 1] = np.mod(pos_wrapped[:, 1], self.config.HEIGHT)
        return pos_wrapped
    
    def compute_separation(self):
        """Principe 1 : Séparation - Éviter les collisions"""
        sep_force = np.zeros((self.config.N, 2))
        
        for i in range(self.config.N):
            count = 0
            steer = np.zeros(2)
            
            for j in range(self.config.N):
                if i != j:
                    diff = self.pos[i] - self.pos[j]
                    dist = np.linalg.norm(diff)
                    if 0 < dist < self.config.DESIRED_SEPARATION:
                        if self.interaction_mode == "quadratic":
                            steer += (diff / dist**3)
                        else:
                            steer += (diff / dist)
                        count += 1
            if count > 0:
                steer /= count
            sep_force[i] = self._limit_vec(steer, self.config.MAX_FORCE)
        
        return sep_force
    
    def compute_alignment(self):
        """Principe 2 : Alignement - Synchroniser les vitesses"""
        ali_force = np.zeros((self.config.N, 2))
        
        for i in range(self.config.N):
            count = 0
            avg_vel = np.zeros(2)
            for j in range(self.config.N):
                if i != j:
                    dist = np.linalg.norm(self.pos[i] - self.pos[j])
                    if 0 < dist < self.config.ALIGNMENT_RADIUS:
                        avg_vel += self.vel[j]
                        count += 1
            if count > 0:
                avg_vel /= count
                norm = np.linalg.norm(avg_vel)
                if norm > 0:
                    desired = (avg_vel / norm) * self.config.MAX_SPEED
                    steer = desired - self.vel[i]
                    ali_force[i] = self._limit_vec(steer, self.config.MAX_FORCE)
        
        return ali_force
    
    def compute_cohesion(self):
        """Principe 3 : Cohésion - Se rapprocher du groupe"""
        coh_force = np.zeros((self.config.N, 2))
        
        for i in range(self.config.N):
            count = 0
            center_mass = np.zeros(2)
            for j in range(self.config.N):
                if i != j:
                    dist = np.linalg.norm(self.pos[i] - self.pos[j])
                    if 0 < dist < self.config.NEIGHBOR_RADIUS:
                        center_mass += self.pos[j]
                        count += 1
            if count > 0:
                center_mass /= count
                desired = center_mass - self.pos[i]
                norm = np.linalg.norm(desired)
                desired = (desired / norm) * self.config.MAX_SPEED
                steer = desired - self.vel[i]
                coh_force[i] = self._limit_vec(steer, self.config.MAX_FORCE)
        
        return coh_force
    
    def update(self, sep_force, ali_force, coh_force):
        """Mise à jour position et vitesse avec intégration"""
        force = (self.config.W_SEPARATION * sep_force +
                self.config.W_ALIGNMENT * ali_force +
                self.config.W_COHESION * coh_force)
        self.vel += force * self.config.dt
        self.vel = self._limit_vec(self.vel, self.config.MAX_SPEED)
        self.pos += self.vel * self.config.dt
        self.pos = self._wrap_position(self.pos)
    
    def step(self):
        """Effectuer une étape de simulation"""
        sep_force = self.compute_separation()
        ali_force = self.compute_alignment()
        coh_force = self.compute_cohesion()
        self.update(sep_force, ali_force, coh_force)
        return sep_force, ali_force, coh_force




In [ ]:
# Plot démo NumPy: positions finales pour chaque configuration
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, cfg_name in enumerate(["cohesive", "fragmented", "chaotic"]):
    sim_demo = BoidSimulationNumPy(CONFIGS[cfg_name], interaction_mode="linear")
    for _ in range(150):
        sim_demo.step()
    pos_final = sim_demo.pos
    vel_final = sim_demo.vel

    ax = axes[idx]
    ax.scatter(pos_final[:, 0], pos_final[:, 1], s=20, alpha=0.7, color=['steelblue','forestgreen','coral'][idx])
    center = np.array([CONFIGS[cfg_name].WIDTH * 0.5, CONFIGS[cfg_name].HEIGHT * 0.5])
    R = min(CONFIGS[cfg_name].WIDTH, CONFIGS[cfg_name].HEIGHT) * 0.5 * 0.98
    circle = plt.Circle(center, R, fill=False, color='black', linestyle='--', alpha=0.4, linewidth=1)
    ax.add_patch(circle)
    ax.set_aspect('equal')
    ax.set_xlim(-1, CONFIGS[cfg_name].WIDTH + 1)
    ax.set_ylim(-1, CONFIGS[cfg_name].HEIGHT + 1)
    ax.set_title(f"NumPy - {cfg_name}")
    ax.grid(True, alpha=0.2)

    # Flèches (quiver) pour les vitesses (sous-échantillon pour lisibilité)
    step_q = max(1, CONFIGS[cfg_name].N // 60)  # ~60 flèches max
    sample_idx = np.arange(0, CONFIGS[cfg_name].N, step_q)
    ax.quiver(pos_final[sample_idx, 0], pos_final[sample_idx, 1],
              vel_final[sample_idx, 0], vel_final[sample_idx, 1],
              angles='xy', scale_units='xy', scale=1.0, color='gray', alpha=0.7, width=0.003)

plt.tight_layout()
plt.show()

## Étape 3 : Ajout des Obstacles (Mode Constant et Quadratique)

In [ ]:
@dataclass
class Obstacle:
    """Représentation d'un obstacle circulaire"""
    x: float  # Position X du centre
    y: float  # Position Y du centre
    radius: float  # Rayon physique de l'obstacle
    influence_radius: float = 5.0  # Rayon de la zone d'influence pour la répulsion

class BoidSimulationWithObstacles(BoidSimulationNumPy):
    """Extension avec support des obstacles - hérite de BoidSimulationNumPy"""
    
    def __init__(self, config: SimulationConfig, 
                 obstacles: List[Obstacle] = None,
                 obstacle_mode: str = "constant"):
        super().__init__(config, interaction_mode="linear")
        self.obstacles = obstacles if obstacles else []
        # Mode d'évitement : "constant" ou "quadratic" (1/r²)
        self.obstacle_mode = obstacle_mode
    
    def compute_obstacle_avoidance(self):
        """Calcul de la répulsion d'obstacles"""
        obs_force = np.zeros((self.config.N, 2))
        
        for i in range(self.config.N):
            force = np.zeros(2)
            
            # Pour chaque obstacle
            for obs in self.obstacles:
                # Vecteur depuis l'obstacle vers le boid
                dx = self.pos[i, 0] - obs.x
                dy = self.pos[i, 1] - obs.y
                d = np.sqrt(dx**2 + dy**2)
                
                # Distance depuis la surface de l'obstacle
                dist_from_surface = d - obs.radius
                
                # Si le boid est dans la zone d'influence
                if dist_from_surface > 0 and d < obs.influence_radius:
                    # Vecteur direction (normalisation)
                    direction = np.array([dx / d, dy / d])
                    
                    if self.obstacle_mode == "quadratic":
                        # Mode quadratique : 1/r² (répulsion très forte près de l'obstacle)
                        magnitude = 1.0 / (dist_from_surface**2 + 0.01)
                    else:
                        # Mode constant : décroissance linéaire de la zone d'influence
                        magnitude = 1.0 - (dist_from_surface / (obs.influence_radius - obs.radius))
                    
                    # Accumuler la force répulsive
                    force += direction * magnitude
            
            # Limiter et appliquer une amplification (5x MAX_FORCE)
            if np.linalg.norm(force) > 0:
                obs_force[i] = self._limit_vec(force, 1.0) * self.config.MAX_FORCE * 5.0
        
        return obs_force
    
    def step(self):
        """Une étape de simulation avec obstacles"""
        # Calculer les trois forces de Reynolds
        sep_force = self.compute_separation()
        ali_force = self.compute_alignment()
        coh_force = self.compute_cohesion()
        # Calculer la force d'évitement d'obstacles
        obs_force = self.compute_obstacle_avoidance()
        
        # Intégrer toutes les forces
        force = (self.config.W_SEPARATION * sep_force +
                self.config.W_ALIGNMENT * ali_force +
                self.config.W_COHESION * coh_force +
                obs_force)  # Obstacles sans poids supplémentaire (déjà amplifiés)
        
        # Mise à jour position et vitesse
        self.vel += force * self.config.dt
        self.vel = self._limit_vec(self.vel, self.config.MAX_SPEED)
        self.pos += self.vel * self.config.dt
        self.pos = self._wrap_position(self.pos)
        
        return sep_force, ali_force, coh_force, obs_force



In [ ]:
# Plot démo Obstacles (NumPy): positions finales avec obstacles
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

# Relancer une simulation pour visualiser
obstacles_demo = [
    Obstacle(x=17.5, y=17.5, radius=2.0, influence_radius=5.0),
    Obstacle(x=30.0, y=15.0, radius=1.5, influence_radius=4.0),
    Obstacle(x=10.0, y=28.0, radius=1.0, influence_radius=4.5),
 ]
sim_obs_demo = BoidSimulationWithObstacles(CONFIGS["cohesive"], obstacles=obstacles_demo, obstacle_mode="quadratic")
for _ in range(150):
    sim_obs_demo.step()
pos_final = sim_obs_demo.pos
vel_final = sim_obs_demo.vel

# Points des boids
ax.scatter(pos_final[:, 0], pos_final[:, 1], s=25, alpha=0.7, color='steelblue', label='Boids')

# Domaine circulaire
center = np.array([CONFIGS["cohesive"].WIDTH * 0.5, CONFIGS["cohesive"].HEIGHT * 0.5])
R = min(CONFIGS["cohesive"].WIDTH, CONFIGS["cohesive"].HEIGHT) * 0.5 * 0.98
circle = plt.Circle(center, R, fill=False, color='black', linestyle='--', alpha=0.4, linewidth=1)
ax.add_patch(circle)

# Obstacles (disque + influence)
for obs in obstacles_demo:
    obs_circle = plt.Circle((obs.x, obs.y), obs.radius, fill=True, color='red', alpha=0.6)
    ax.add_patch(obs_circle)
    inf_circle = plt.Circle((obs.x, obs.y), obs.influence_radius, fill=False, color='red', linestyle='--', alpha=0.3, linewidth=1)
    ax.add_patch(inf_circle)

# Flèches (quiver) pour les vitesses
step_q = max(1, CONFIGS["cohesive"].N // 60)
sample_idx = np.arange(0, CONFIGS["cohesive"].N, step_q)
ax.quiver(pos_final[sample_idx, 0], pos_final[sample_idx, 1],
          vel_final[sample_idx, 0], vel_final[sample_idx, 1],
          angles='xy', scale_units='xy', scale=1.0, color='gray', alpha=0.8, width=0.003)

ax.set_aspect('equal')
ax.set_xlim(-1, CONFIGS["cohesive"].WIDTH + 1)
ax.set_ylim(-1, CONFIGS["cohesive"].HEIGHT + 1)
ax.set_title("NumPy + Obstacles (Quadratique)")
ax.grid(True, alpha=0.2)
ax.legend()

plt.tight_layout()
plt.show()

## Étape 4 : Calcul de la Vitesse d'Ensemble $V_{A}$

La vitesse d'ensemble V_A est définie comme la norme de la moyenne des vitesses individuelles :

$$
V_A(t) = \frac{\left\| \sum_{i=1}^{N} \vec{v}_i(t) \right\|}{\sum_{i=1}^{N} \left\| \vec{v}_i(t) \right\|}
$$

In [ ]:
def run_simulation_numpy(sim: BoidSimulationNumPy, steps: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Exécute une simulation et retourne V_A(t) et historique des positions
    
    V_A(t) = ||Σ v_i(t)|| / Σ ||v_i(t)||  (paramètre d'ordre, [0,1])
    """
    va_history = []
    pos_history = []
    
    for t in range(steps):
        # Effectuer une étape de simulation
        sim.step()
        
        # Calcul V_A : ||Σ v_i|| / Σ ||v_i||
        sum_v = np.sum(sim.vel, axis=0)
        sum_norms = np.sum(np.linalg.norm(sim.vel, axis=1))
        va = (np.linalg.norm(sum_v) / sum_norms) if sum_norms > 1e-12 else 0.0
        va_history.append(va)
        
        # Stocker l'état pour visualisation
        pos_history.append(sim.pos.copy())
    
    return np.array(va_history), np.array(pos_history)

# Simulation test sur la configuration cohésive (200 steps pour démonstration)
print("Exécution simulation NumPy (cohésive, 200 steps)...")
sim_test = BoidSimulationNumPy(CONFIGS["cohesive"], interaction_mode="linear")
va_test, pos_test = run_simulation_numpy(sim_test, 200)

# Créer la figure avec 2 sous-graphiques
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1 : Évolution de V_A(t)
ax1.plot(va_test, linewidth=2, color='steelblue')
ax1.set_xlabel('Temps (itérations)')
ax1.set_ylabel("Paramètre d'ordre V_A")
ax1.set_title('Évolution de V_A (||Σv|| / Σ||v||) - Cohésive')
ax1.grid(True, alpha=0.3)

# Graphique 2 : Positions finales
pos_final = pos_test[-1]  # Positions au dernier pas de temps

## Étape 5 : Implémentation Taichi (GPU)

Taichi est un framework de calcul haute-performance qui compile automatiquement vers GPU.
Nous allons réimplémenter le même modèle pour la performance.

In [ ]:
# Implémentation complète de BoidSimulationTaichi (GPU)
ti.init(arch=ti.gpu)

@ti.data_oriented
class BoidSimulationTaichi:
    def __init__(self, config, obstacles=None, obstacle_mode: str = "constant"):
        # Extraire les paramètres de la config
        cfg = config
        self.cfg = cfg
        self.n = cfg.N
        self.width = cfg.WIDTH
        self.height = cfg.HEIGHT
        self.obstacle_mode = obstacle_mode
        
        # Champs de position/vitesse/accélération
        self.pos = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.vel = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.acc = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.sep_force = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.ali_force = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.coh_force = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        self.obs_force = ti.Vector.field(2, dtype=ti.f32, shape=self.n)
        
        # Initialiser les forces à zéro
        self._init_forces()
        
        # Initialiser les positions/vitesses
        self._init_positions()
        
        # Obstacles
        self.obstacles = obstacles if obstacles else []
        self.num_obs = len(self.obstacles)
        if self.num_obs > 0:
            self.obs_x = ti.field(ti.f32, self.num_obs)
            self.obs_y = ti.field(ti.f32, self.num_obs)
            self.obs_rad = ti.field(ti.f32, self.num_obs)
            self.obs_influence = ti.field(ti.f32, self.num_obs)
            for i, obs in enumerate(self.obstacles):
                self.obs_x[i] = obs.x
                self.obs_y[i] = obs.y
                self.obs_rad[i] = obs.radius
                self.obs_influence[i] = obs.influence_radius
        else:
            self.obs_x = ti.field(ti.f32, 1)
            self.obs_y = ti.field(ti.f32, 1)
            self.obs_rad = ti.field(ti.f32, 1)
            self.obs_influence = ti.field(ti.f32, 1)
    
    @ti.kernel
    def _init_forces(self):
        for i in range(self.n):
            self.sep_force[i] = ti.Vector([0.0, 0.0])
            self.ali_force[i] = ti.Vector([0.0, 0.0])
            self.coh_force[i] = ti.Vector([0.0, 0.0])
            self.obs_force[i] = ti.Vector([0.0, 0.0])
    
    @ti.kernel
    def _init_positions(self):
        center = ti.Vector([self.width * 0.5, self.height * 0.5])
        R = ti.min(self.width, self.height) * 0.5 * 0.98
        for i in range(self.n):
            r = ti.sqrt(ti.random()) * R
            theta = ti.random() * 2.0 * ti.math.pi
            offset = ti.Vector([ti.cos(theta), ti.sin(theta)]) * r
            self.pos[i] = center + offset
            ang = ti.random() * 2.0 * ti.math.pi
            self.vel[i] = ti.Vector([ti.cos(ang), ti.sin(ang)])
            self.acc[i] = ti.Vector([0.0, 0.0])
    
    @ti.kernel
    def _sep_kernel(self):
        for i in range(self.n):
            steer = ti.Vector([0.0, 0.0])
            c = 0
            for j in range(self.n):
                if i != j:
                    dvec = self.pos[i] - self.pos[j]
                    d = dvec.norm()
                    if 0.0 < d < self.cfg.DESIRED_SEPARATION:
                        steer += dvec / (d**3 + 1e-6)
                        c += 1
            if c > 0:
                steer *= 1.0 / c
            self.sep_force[i] = self._limit_vec(steer, self.cfg.MAX_FORCE)
    
    @ti.kernel
    def _ali_kernel(self):
        for i in range(self.n):
            avg = ti.Vector([0.0, 0.0])
            c = 0
            for j in range(self.n):
                if i != j:
                    d = (self.pos[i] - self.pos[j]).norm()
                    if 0.0 < d < self.cfg.ALIGNMENT_RADIUS:
                        avg += self.vel[j]
                        c += 1
            steer = ti.Vector([0.0, 0.0])
            if c > 0:
                avg *= 1.0 / c
                n = avg.norm()
                if n > 1e-6:
                    desired = avg / n * self.cfg.MAX_SPEED
                    steer = desired - self.vel[i]
            self.ali_force[i] = self._limit_vec(steer, self.cfg.MAX_FORCE)
    
    @ti.kernel
    def _coh_kernel(self):
        for i in range(self.n):
            center = ti.Vector([0.0, 0.0])
            c = 0
            for j in range(self.n):
                if i != j:
                    d = (self.pos[i] - self.pos[j]).norm()
                    if 0.0 < d < self.cfg.NEIGHBOR_RADIUS:
                        center += self.pos[j]
                        c += 1
            steer = ti.Vector([0.0, 0.0])
            if c > 0:
                center *= 1.0 / c
                desired = center - self.pos[i]
                n = desired.norm()
                if n > 1e-6:
                    desired = desired / n * self.cfg.MAX_SPEED
                    steer = desired - self.vel[i]
            self.coh_force[i] = self._limit_vec(steer, self.cfg.MAX_FORCE)
    
    @ti.kernel
    def _obs_kernel(self):
        for i in range(self.n):
            force = ti.Vector([0.0, 0.0])
            for k in range(self.num_obs):
                dx = self.pos[i][0] - self.obs_x[k]
                dy = self.pos[i][1] - self.obs_y[k]
                d = ti.sqrt(dx*dx + dy*dy)
                
                if d < self.obs_influence[k] and d > self.obs_rad[k]:
                    dist_surface = d - self.obs_rad[k]
                    direction = ti.Vector([dx, dy]) / d
                    
                    if self.obstacle_mode == "quadratic":
                        magnitude = 1.0 / (dist_surface**2 + 0.01)
                    else:
                        magnitude = 1.0 - (dist_surface / (self.obs_influence[k] - self.obs_rad[k]))
                    
                    force += direction * magnitude
            
            self.obs_force[i] = self._limit_vec(force, self.cfg.MAX_FORCE) * 5.0
    
    @ti.kernel
    def _update_kernel(self):
        for i in range(self.n):
            f_sep = self.cfg.W_SEPARATION * self.sep_force[i]
            f_ali = self.cfg.W_ALIGNMENT * self.ali_force[i]
            f_coh = self.cfg.W_COHESION * self.coh_force[i]
            force = f_sep + f_ali + f_coh + self.obs_force[i]
            
            self.vel[i] = self._limit_vec(self.vel[i] + force * self.cfg.dt, self.cfg.MAX_SPEED)
            self.pos[i] += self.vel[i] * self.cfg.dt
            
            # Wrap circulaire (antipode)
            cx = self.width * 0.5
            cy = self.height * 0.5
            R = ti.min(self.width, self.height) * 0.5 * 0.98
            dx = self.pos[i][0] - cx
            dy = self.pos[i][1] - cy
            r = ti.sqrt(dx*dx + dy*dy)
            
            if r > R:
                scale = R / r
                self.pos[i][0] = cx - dx * scale
                self.pos[i][1] = cy - dy * scale
    
    @ti.func
    def _limit_vec(self, v, max_val):
        n = v.norm()
        res = v
        if n > max_val:
            res = v / n * max_val
        return res
    
    def step(self):
        """Effectuer une étape de simulation complète"""
        self._sep_kernel()
        self._ali_kernel()
        self._coh_kernel()
        if self.num_obs > 0:
            self._obs_kernel()
        self._update_kernel()
    
    def get_va(self):
        """Calculer la vitesse d'ensemble V_A"""
        vel_np = self.vel.to_numpy()
        sum_v = np.sum(vel_np, axis=0)
        sum_norms = np.sum(np.linalg.norm(vel_np, axis=1))
        va = np.linalg.norm(sum_v) / sum_norms if sum_norms > 1e-12 else 0.0
        return float(va)


Fonction pour lancer une animation des boids depuis le notebook avec posibilité de sauvegarde

In [ ]:
# Animation Taichi (sans flèches par défaut, T depuis config)
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, display
import matplotlib as mpl
import os

mpl.rcParams["animation.embed_limit"] = 30_000_000

def animate_taichi(config_key: str = "cohesive", steps: int = None, interval: int = 40, show_quiver: bool = False, save_gif_path: str = None):
    """
    Anime la simulation Taichi pour une configuration donnée.
    - steps: si None, utilise cfg.T défini dans CONFIGS (ex. 1500)
    - show_quiver: False par défaut (pas de flèches)
    """
    cfg = CONFIGS[config_key]
    if steps is None:
        steps = cfg.T
    sim = BoidSimulationTaichi(cfg, obstacle_mode="constant")

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))

    center = np.array([cfg.WIDTH * 0.5, cfg.HEIGHT * 0.5])
    R = min(cfg.WIDTH, cfg.HEIGHT) * 0.5 * 0.98
    circle = plt.Circle(center, R, fill=False, color='black', linestyle='--', alpha=0.4, linewidth=1)
    ax.add_patch(circle)

    pos_np = sim.pos.to_numpy()
    scat = ax.scatter(pos_np[:, 0], pos_np[:, 1], s=24, alpha=0.7, color='steelblue')

    quiv = None
    sample_step = max(1, cfg.N // 80)
    sample_idx = np.arange(0, cfg.N, sample_step)
    if show_quiver:
        vel_np = sim.vel.to_numpy()
        quiv = ax.quiver(pos_np[sample_idx, 0], pos_np[sample_idx, 1],
                         vel_np[sample_idx, 0], vel_np[sample_idx, 1],
                         angles='xy', scale_units='xy', scale=1.0, color='gray', alpha=0.7, width=0.003)

    ax.set_xlim(-1, cfg.WIDTH + 1)
    ax.set_ylim(-1, cfg.HEIGHT + 1)
    ax.set_aspect('equal')
    ax.set_title(f"Taichi Animation - {cfg.name}")
    ax.grid(True, alpha=0.2)

    def update(frame):
        sim.step()
        pos = sim.pos.to_numpy()
        scat.set_offsets(pos)
        if show_quiver and quiv is not None:
            vel = sim.vel.to_numpy()
            quiv.set_offsets(pos[sample_idx])
            quiv.set_UVC(vel[sample_idx, 0], vel[sample_idx, 1])
        return scat if quiv is None else (scat, quiv)

    anim = FuncAnimation(fig, update, frames=steps, interval=interval, blit=False)

    if save_gif_path:
        try:
            os.makedirs(os.path.dirname(save_gif_path), exist_ok=True)
        except Exception:
            pass
        writer = PillowWriter(fps=max(1, int(1000/interval)))
        anim.save(save_gif_path, writer=writer)
        print(f"✓ GIF sauvegardé: {save_gif_path}")

    out = HTML(anim.to_jshtml())
    display(out)
    plt.close(fig)
    return out

print("Animations interactives prêtes. Exemples:")
print(" - animate_taichi('cohesive')  # utilise T de la config")
print(" - animate_taichi('fragmented')")
print(" - animate_taichi('chaotic')")

Lancer l'animation depuis cette cellule directement

In [ ]:
# Lancer directement une animation Taichi en temps réel (exécution immédiate)
_ = animate_taichi('cohesive')

In [ ]:
# Plot démo Taichi: V_A(t) et positions finales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Relancer une démo avec T de la configuration
cfg = CONFIGS["cohesive"]
sim_taichi_plot = BoidSimulationTaichi(cfg, obstacle_mode="constant")
va_t = []
for _ in range(cfg.T):
    sim_taichi_plot.step()
    va_t.append(sim_taichi_plot.get_va())

# Courbe V_A(t)
ax1.plot(va_t, linewidth=2, color='coral')
ax1.set_xlabel('Temps (itérations)')
ax1.set_ylabel('V_A')
ax1.set_title('Taichi - Évolution de V_A')
ax1.grid(True, alpha=0.3)

# Positions finales (sans flèches)
pos_final_gpu = sim_taichi_plot.pos.to_numpy()
ax2.scatter(pos_final_gpu[:, 0], pos_final_gpu[:, 1], s=25, alpha=0.7, color='coral')

center = np.array([cfg.WIDTH * 0.5, cfg.HEIGHT * 0.5])
R = min(cfg.WIDTH, cfg.HEIGHT) * 0.5 * 0.98
circle = plt.Circle(center, R, fill=False, color='black', linestyle='--', alpha=0.4, linewidth=1)
ax2.add_patch(circle)
ax2.set_aspect('equal')
ax2.set_xlim(-1, cfg.WIDTH + 1)
ax2.set_ylim(-1, cfg.HEIGHT + 1)
ax2.set_title('Taichi - Positions finales')
ax2.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
# Contrôles interactifs pour lancer et sauvegarder l'animation Taichi
import ipywidgets as widgets
from IPython.display import display

# Sélecteurs
cfg_dropdown = widgets.Dropdown(
    options=[('Cohésive', 'cohesive'), ('Fragmentée', 'fragmented'), ('Chaotique', 'chaotic')],
    value='cohesive',
    description='Config:'
)

steps_slider = widgets.IntSlider(value=600, min=100, max=2000, step=50, description='Steps:', continuous_update=False)
interval_slider = widgets.IntSlider(value=40, min=10, max=120, step=5, description='Interval (ms):', continuous_update=False)
quiver_toggle = widgets.Checkbox(value=True, description='Afficher flèches')

save_toggle = widgets.Checkbox(value=False, description='Sauvegarder GIF')
fname_text = widgets.Text(value='animations/taichi_animation.gif', description='Fichier:')

# Bouton de lancement
run_button = widgets.Button(description='Lancer animation', button_style='primary')
out = widgets.Output()

def on_run_clicked(b):
    with out:
        out.clear_output()
        save_path = fname_text.value if save_toggle.value else None
        print(f"→ Animation: {cfg_dropdown.label} (steps={steps_slider.value}, interval={interval_slider.value}, quiver={quiver_toggle.value})")
        if save_path:
            print(f"→ Sauvegarde GIF vers: {save_path}")
        animate_taichi(cfg_dropdown.value, steps=steps_slider.value, interval=interval_slider.value,
                       show_quiver=quiver_toggle.value, save_gif_path=save_path)

run_button.on_click(on_run_clicked)

ui = widgets.VBox([
    widgets.HBox([cfg_dropdown, steps_slider, interval_slider, quiver_toggle]),
    widgets.HBox([save_toggle, fname_text]),
    run_button,
    out
])

display(ui)

In [ ]:
# Plot Taichi avec obstacles: positions finales pour 2 modes
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

obstacles_taichi = [
    Obstacle(x=17.5, y=17.5, radius=2.0, influence_radius=5.0),
    Obstacle(x=12.0, y=26.0, radius=1.2, influence_radius=4.2)
]

for idx, mode in enumerate(["constant", "quadratic"]):
    sim_obs_gpu = BoidSimulationTaichi(CONFIGS["cohesive"], obstacles=obstacles_taichi, obstacle_mode=mode)
    for _ in range(180):
        sim_obs_gpu.step()
    pos_final = sim_obs_gpu.pos.to_numpy()
    
    ax = axes[idx]
    ax.scatter(pos_final[:, 0], pos_final[:, 1], s=28, alpha=0.7, color=['steelblue','coral'][idx])
    
    center = np.array([CONFIGS["cohesive"].WIDTH * 0.5, CONFIGS["cohesive"].HEIGHT * 0.5])
    R = min(CONFIGS["cohesive"].WIDTH, CONFIGS["cohesive"].HEIGHT) * 0.5 * 0.98
    circle = plt.Circle(center, R, fill=False, color='black', linestyle='--', alpha=0.4, linewidth=1)
    ax.add_patch(circle)
    
    for obs in obstacles_taichi:
        obs_circle = plt.Circle((obs.x, obs.y), obs.radius, fill=True, color='red', alpha=0.6)
        ax.add_patch(obs_circle)
        inf_circle = plt.Circle((obs.x, obs.y), obs.influence_radius, fill=False, color='red', linestyle='--', alpha=0.3, linewidth=1)
        ax.add_patch(inf_circle)
    
    ax.set_aspect('equal')
    ax.set_xlim(-1, CONFIGS["cohesive"].WIDTH + 1)
    ax.set_ylim(-1, CONFIGS["cohesive"].HEIGHT + 1)
    ax.set_title(f"Taichi + Obstacles ({mode})")
    ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

## Étape 7 : Analyse Multi-Configuration

In [ ]:
# Exécuter la simulation avec les trois configurations
print("Exécution sur les 3 configurations (Taichi, 200 steps chacune)...\n")

results = {}
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for idx, (config_name, config) in enumerate(CONFIGS.items()):
    print(f"  {config_name.upper()}...", end=" ", flush=True)
    
    # Créer et lancer simulation Taichi
    sim = BoidSimulationTaichi(config, obstacle_mode="constant")
    va_list = []
    
    # Boucle de simulation
    for t in range(2000):
        sim.step()
        va_list.append(sim.get_va())
    
    va_array = np.array(va_list)
    results[config_name] = va_array
    
    # Visualiser V_A(t) pour cette configuration
    axes[idx].plot(va_array, linewidth=2, color=['steelblue', 'forestgreen', 'coral'][idx])
    axes[idx].set_xlabel('Temps (itérations)')
    axes[idx].set_ylabel('V_A')
    axes[idx].set_title(f'{config.name}\n(N={config.N}, densité={config.N/(config.WIDTH*config.HEIGHT):.2f})')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([0, max(va_array) * 1.1])
    
    # Afficher statistiques
    print(f"✓ V_A: {np.mean(va_array):.4f} ± {np.std(va_array):.4f}")

plt.tight_layout()
plt.show()

# Afficher tableau comparatif
print(f"\n{'='*70}")
print(f"TABLEAU COMPARATIF - Vitesse d'ensemble par configuration")
print(f"{'='*70}")
print(f"{'Config':<15} {'N':<6} {'Densité':<10} {'V_A moy':<12} {'V_A std':<12}")
print(f"{'-'*70}")
for config_name, config in CONFIGS.items():
    va = results[config_name]
    density = config.N / (config.WIDTH * config.HEIGHT)
    # densité = nombre de boids par unité carrée
    print(f"{config_name:<15} {config.N:<6} {density:<10.3f} {np.mean(va):<12.4f} {np.std(va):<12.4f}")

## Étape 8 : Test des Obstacles (Mode Constant vs Quadratique)

In [ ]:
# Définir un obstacle pour les tests de comparaison
obstacles_test = [
    Obstacle(x=17.5, y=17.5, radius=2.0, influence_radius=5.0)
]

print("Test obstacles: comparaison mode constant vs quadratique\n")

# Mode d'évitement CONSTANT
print("  Mode CONSTANT...", end=" ", flush=True)
sim_const = BoidSimulationTaichi(CONFIGS["cohesive"], 
                                 obstacles=obstacles_test,
                                 obstacle_mode="constant")
va_const = []
for t in range(150):
    sim_const.step()
    va_const.append(sim_const.get_va())
va_const = np.array(va_const)
print(f"✓ V_A moy: {np.mean(va_const):.4f}")

# Mode d'évitement QUADRATIQUE (1/r²)
print("  Mode QUADRATIQUE...", end=" ", flush=True)
sim_quad = BoidSimulationTaichi(CONFIGS["cohesive"],
                               obstacles=obstacles_test,
                               obstacle_mode="quadratic")
va_quad = []
for t in range(150):
    sim_quad.step()
    va_quad.append(sim_quad.get_va())
va_quad = np.array(va_quad)
print(f"✓ V_A moy: {np.mean(va_quad):.4f}")

# Visualiser la comparaison
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Graphique 1 : Mode CONSTANT
axes[0].plot(va_const, linewidth=2, color='steelblue')
axes[0].fill_between(range(len(va_const)), va_const, alpha=0.3, color='steelblue')
axes[0].set_xlabel('Temps (itérations)')
axes[0].set_ylabel('V_A')
axes[0].set_title(f'Obstacles - Mode CONSTANT\nV_A moy: {np.mean(va_const):.4f}')
axes[0].grid(True, alpha=0.3)

# Graphique 2 : Mode QUADRATIQUE
axes[1].plot(va_quad, linewidth=2, color='coral')
axes[1].fill_between(range(len(va_quad)), va_quad, alpha=0.3, color='coral')
axes[1].set_xlabel('Temps (itérations)')
axes[1].set_ylabel('V_A')
axes[1].set_title(f'Obstacles - Mode QUADRATIQUE (1/r²)\nV_A moy: {np.mean(va_quad):.4f}')
axes[1].grid(True, alpha=0.3)

# Graphique 3 : Superposition pour comparaison
axes[2].plot(va_const, linewidth=2, color='steelblue', alpha=0.7, label='Constant')
axes[2].plot(va_quad, linewidth=2, color='coral', alpha=0.7, label='Quadratique')
axes[2].set_xlabel('Temps (itérations)')
axes[2].set_ylabel('V_A')
axes[2].set_title('Comparaison : Constant vs Quadratique')
axes[2].grid(True, alpha=0.3)
axes[2].legend()

plt.tight_layout()
plt.show()

# Analyse statistique
print(f"\n{'='*60}")
print(f"ANALYSE OBSTACLES")
print(f"{'='*60}")
print(f"Mode CONSTANT    : V_A moy = {np.mean(va_const):.4f} ± {np.std(va_const):.4f}")
print(f"Mode QUADRATIQUE : V_A moy = {np.mean(va_quad):.4f} ± {np.std(va_quad):.4f}")
print(f"Différence       : {np.mean(va_quad) - np.mean(va_const):+.4f}")
if np.mean(va_quad) > np.mean(va_const):
    print("→ La répulsion quadratique est PLUS FORTE (bon!)")
else:
    print("→ La répulsion quadratique est MOINS FORTE (à investiguer)")